In [1]:
import os
import numpy as np
import pandas as pd
from tendrils import api, utils

from multiprocessing import Pool

utils.copy_from_other_config('config.ini')
OUT_DIR = 'data'
PROCESSES = 8

In [2]:
targets_list = api.get_targets()
targets_df = pd.DataFrame(targets_list)

flows_df = targets_df[targets_df.project=='flows']
flows_df = flows_df[flows_df.target_status=='target']
# remove targets without z
flows_df = flows_df[flows_df.redshift > 0] 

In [3]:
def extract_target_phot(target_id):
    """Gets the light curves from ZTF (optical) and FLOWS
    website (NIR) for a given FLOWS target.
    
    Parameters
    ----------
    target_id: int
        Target's ID from FLOWS website.
        
    Returns
    -------
    opt_df: DataFrame
        DataFrame with information (mjd, mag, etc.) from the 
        optical light curves.
    nir_df: DataFrame
        DataFrame with information (mjd, mag, etc.) from the 
        NIR light curves.
    """
    # Opt
    taget_opt_phot = utils.download_ztf_photometry(target_id)
    opt_df = taget_opt_phot.to_pandas()
    opt_df.rename(columns={'time':'mjd'}, inplace=True) 

    # NIR
    taget_nir_phot = api.get_lightcurve(target_id)
    taget_nir_phot['mjd'] = taget_nir_phot['time'].value
    nir_df = taget_nir_phot.to_pandas()
    nir_df.rename(columns={'mag_raw':'mag', 
                           'mag_raw_error':'mag_err'}, 
                  inplace=True)
    
    return opt_df, nir_df

In [4]:
def pool_extract(target_name):
    """Creates a light-curve file with SNooPy format.
    
    target_name: str
        Target's name as given by FLOWS.
    """
    target_df = flows_df[flows_df.target_name==target_name]
    target_id = target_df.targetid.values[0]
    
    try:
        # photometry
        opt_df, nir_df = extract_target_phot(target_id)

        # metadata for SNooPy
        ra = target_df.ra.values[0]
        dec = target_df.decl.values[0]
        z = target_df.redshift.values[0]
        header = f'{target_name} {z} {ra} {dec}\n'

        outfile = os.path.join(OUT_DIR, f'{target_name}_snpy.txt')
        with open(outfile, 'w') as file:
            file.write(header)

            # NIR
            for filt in nir_df.photfilter.unique():
                if filt in ['J', 'H']:
                    # temporary patch
                    file.write(f'filter {filt}\n')
                    filt_df = nir_df[nir_df.photfilter==filt]

                    for mjd, mag, mag_err in zip(filt_df.mjd, 
                                                 filt_df.mag, 
                                                 filt_df.mag_err):
                        file.write(f' {mjd} {mag} {mag_err}\n')

            # Opt
            for filt in opt_df.photfilter.unique():
                file.write(f'filter {filt[0]}\n')
                filt_df = opt_df[opt_df.photfilter==filt]

                for mjd, mag, mag_err in zip(filt_df.mjd, 
                                             filt_df.mag, 
                                             filt_df.mag_err):
                    file.write(f' {mjd} {mag} {mag_err}\n')
    except Exception as message:
        print(f'{target_name} failed: {message}')

In [5]:
results = Pool(PROCESSES).map(pool_extract, (target_name for target_name \
                                             in flows_df.target_name))

2021fg failed: 'NoneType' object has no attribute 'to_pandas'
2021cp failed: 404 Client Error: Not Found for url: https://flows.phys.au.dk/api/lightcurve.php?target=415
2020uvd failed: 404 Client Error: Not Found for url: https://flows.phys.au.dk/api/lightcurve.php?target=29
2020aakx failed: 'NoneType' object has no attribute 'to_pandas'
2020zka failed: 404 Client Error: Not Found for url: https://flows.phys.au.dk/api/lightcurve.php?target=232
2020adle failed: 'NoneType' object has no attribute 'to_pandas'
2021zw failed: 'NoneType' object has no attribute 'to_pandas'
2021our failed: 'NoneType' object has no attribute 'to_pandas'
2021omp failed: 'NoneType' object has no attribute 'to_pandas'
2021bnd failed: 'NoneType' object has no attribute 'to_pandas'
2021pby failed: 'NoneType' object has no attribute 'to_pandas'
2021pfs failed: 'NoneType' object has no attribute 'to_pandas'
2021bsf failed: 'NoneType' object has no attribute 'to_pandas'
2021dch failed: 404 Client Error: Not Found for 